In [2]:
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta

In [3]:
#自分のアカウント、トークンをセット
accountID = "101-009-12923052-001"
access_token = '7b8e8782a87ce72d52cd23b2d74772df-13011574ae44dc0db94fc8dac52e4ebc'
api = API(access_token=access_token, environment="practice")

In [4]:
year_months =[
    [2016, 1], [2016, 2], [2016, 3], [2016, 4], [2016, 5], [2016, 6], [2016, 7], [2016, 8], [2016, 9], [2016, 10], [2016, 11], [2016, 12],
    [2017, 1], [2017, 2], [2017, 3], [2017, 4], [2017, 5], [2017, 6], [2017, 7], [2017, 8], [2017, 9], [2017, 10], [2017, 11], [2017, 12],
    [2018, 1], [2018, 2], [2018, 3], [2018, 4], [2018, 5], [2018, 6], [2018, 7], [2018, 8], [2018, 9], [2018, 10], [2018, 11], [2018, 12],
    [2019, 1], [2019, 2], [2019, 3], [2019, 4], [2019, 5], [2019, 6], [2019, 7],
]

In [5]:
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

In [6]:
def oandaJsonToPythonList(JSONRes):
    data = []
    for res in JSONRes['candles']:
        data.append( [
            datetime.datetime.fromisoformat(res['time'][:19]),
            res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
    return data

In [7]:
all_data = []
# year, monthでループ
for year, month in year_months:
    date_from = datetime.datetime(year, month, 1)
    date_to = date_from + relativedelta(months=+1, day=1)

    ret = getCandleDataFromOanda("USD_JPY", api, date_from, date_to, "M10")
    month_data = oandaJsonToPythonList(ret)

    all_data.extend(month_data)

In [8]:
# pandas DataFrameへ変換
df = pd.DataFrame(all_data)
df.columns = ['Datetime', 'Volume', 'Open', 'High', 'Low', 'Close']
df = df.set_index('Datetime')

In [9]:
df.shape

(133486, 5)

In [10]:
df.head(5)

,Volume,Open,High,Low,Close
Datetime,,,,,
2016-01-03 22:00:00,162,120.195,120.235,120.194,120.227
2016-01-03 22:10:00,208,120.226,120.253,120.209,120.236
2016-01-03 22:20:00,333,120.235,120.283,120.233,120.274
2016-01-03 22:30:00,359,120.274,120.304,120.268,120.286
2016-01-03 22:40:00,242,120.288,120.330,120.277,120.313


In [11]:
# CSVファイルへの出力
df.to_csv('../data/USD_JPY_201601-201908_M10.csv')